### Importing necessary libraries (torch, matplotlib)

In [63]:
import torch
import matplotlib.pyplot as plt
import torch.optim as opt
import torch.nn as nn
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Path to training and testing datasets

In [65]:
PATH = "C:/Users/Ato/Documents/Programming/Python/catdog/src/datasets"
train_path = PATH + "/train"
dogs_path = train_path + "/dogs"
cats_path = train_path + "/cats"
test_path = PATH + "/test1"

## Preprocessing and Observations

In [68]:
print(len(os.listdir(dogs_path)))
print(len(os.listdir(cats_path)))




12500
12500
